<H2>Part 1: Create basic dataframe with Neighbourhoods</H2>

<b>Import all libraries:</b>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

<b>Scrape webpage using Beautiful Soup:</b>

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
html = pd.read_html(str(table))
json = html[0].to_json(orient='records')
neighbors = pd.read_json(json)
neighbors = neighbors [["Postcode", "Borough", "Neighbourhood"]]
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b>Clean not assigned Boroughs:</b>

In [3]:
neighbors.drop(neighbors.loc[neighbors['Borough']=="Not assigned"].index, inplace=True)
neighbors = neighbors.reset_index(drop=True)
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<b>Group by Postcode separating Neighbourhoods by comma:</b>

In [4]:
neighbors = neighbors.groupby('Postcode').agg({'Borough' : 'first', 'Neighbourhood' : ', '.join}).reset_index().reindex(columns=neighbors.columns)
neighbors.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Rows with "Not assigned" Neighbourhoods will have the value of the Borough</b>

In [ ]:
neighbors["Neighbourhood"].replace("Not assigned",neighbors["Borough"],inplace=True)
neighbors.loc[neighbors['Postcode']=='M9A'] #Check the replaced row

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


<b>Check the final dataframe:</b>

In [ ]:
neighbors

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<b>Check the shape of the dataframe:</b>

In [ ]:
neighbors.shape

(103, 3)

<H2>Part 2: Get latitud and longitud of each neighbourhood</H2>

<b>Install and Import libraries:</b>

<i>Note: I did not use the geocoder python package, it would not work for me. I used an alternate library called "pgeocode". Final values could differ slightly from other works done using the original library.</i>

In [ ]:
!pip install pgeocode
import pgeocode

<b>Define a function to get coordinates:</b>

In [ ]:
def get_coordinates(pcode):
    geodata = pgeocode.Nominatim('ca')
    data = geodata.query_postal_code(pcode)
    lat = data.latitude
    long = data.longitude
    return lat,long

<b>Get coordinates for each row of the dataframe:</b>

In [ ]:
neighbors['Latitude'], neighbors['Longitude'] = 0.0,0.0

for i in range(0,len(neighbors)):
    neighbors['Latitude'].loc[i],neighbors['Longitude'].loc[i] = get_coordinates(neighbors.iloc[i][0])

<b>Drop values not found:</b>

In [ ]:
neighbors.dropna(subset=['Latitude', 'Longitude'], inplace = True)
neighbors

<H2>Part 3: Exploring and Clustering Neighborhooods</H2>

<b>Import libraries:</b>

In [ ]:
!pip install folium

import folium # map rendering library

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<b>Get only boroughs with the word Toronto:</b>

In [ ]:
neighbors = neighbors[neighbors['Borough'].str.contains("Toronto")]
neighbors = neighbors.reset_index(drop=True)
neighbors

<b>Create a map of the area:</b>

In [ ]:
map_toronto = folium.Map(location=["43.707454", "-79.340665"], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbors['Latitude'], neighbors['Longitude'], neighbors['Borough'], neighbors['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b>Define Foursquare Credentials(hidden):</b>

In [ ]:
# The code was removed by Watson Studio for sharing.

<b>Define function to get nearby Venues for each neighborhood:</b>

In [ ]:
radius = 500
limit = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Get Dataframe with all venue data:</b>

In [ ]:
toronto_venues = getNearbyVenues(names=neighbors['Neighbourhood'],
                                   latitudes=neighbors['Latitude'],
                                   longitudes=neighbors['Longitude']
                                  )

<b>Check Shape of the Dataframe:</b>

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

<b>Count Venues per Neighbourhood:</b>

In [ ]:
toronto_venues.groupby('Neighborhood').count()

<b>Check unique categories:</b>

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

<b>Analyze each neighborhood:</b>

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

<b>Group rows by neighborhood and take the mean of the frequency of occurrence of each category:</b>

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

<b>Print each neighborhood along with the top 5 most common venues:</b>

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<b>Write a function to sort the venues in descending order:</b>

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Create a new dataframe and display the top 5 venues for each neighborhood:</b>

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

<b>Cluster neighborhoods using k-means:</b>

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<b>Create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood</b>

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
toronto_merged = neighbors

# merge neighborhoods_venues_sorted with neighbors to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

In [ ]:
#toronto_merged['Cluster Labels'] = pd.to_numeric(toronto_merged['Cluster Labels'], errors='coerce')
#toronto_merged

#toronto_merged.dropna(subset=['Cluster Labels'], inplace = True)
#toronto_merged

<b>Visualize the resulting clusters:</b>

In [ ]:
# create map
map_toronto = folium.Map(location=["43.6519", "-79.3874"], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

<b>See each cluster:</b>

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

<H2><i>The insight that can be retrieved from observing the clustering, is that there are two main groups of neighborhoods in Toronto. The ones classified within Cluster 0 (with venues mostly related to Parks, Gyms and Playgrounds) and Cluster 4 (with venues mostly related to Coffee Shops and Restaurants).</H2></i>